In [52]:
import os
import numpy as np
from urllib.parse import urlencode 
import pandas as pd 
from IPython.display import Image, display, HTML
import ipywidgets as widgets 
from ipywidgets import interact, interact_manual 
from chart_studio.plotly import iplot 
from astropy import units as u 
from astropy.coordinates import SkyCoord
from astropy.time import Time, TimeDelta
from astroquery.cadc import Cadc 
from datetime import datetime
from cadcutils import net
import cadctap
from cadctap.core import main_app
from cadctap import CadcTapClient
from astropy.io.votable.tree import VOTableFile, Resource, Table, Field
from astropy.io.votable import from_table, writeto
from astropy.table import Table
from six import  BytesIO
import glue_jupyter as gj


params = {'claimedtype' : 'Ia', 'format' : 'csv' }
Catalog_domain ='https://api.sne.space/catalog'
query_str  = urlencode(params)
url = '{}?{}&first'.format(Catalog_domain,query_str)
sn_data = pd.read_csv(url)

###calculate coords, maxdate, start_date , end_date, and put them into the dataframe as columns

sn_data["ra"] =SkyCoord(sn_data["ra"], sn_data["dec"],unit=(u.hourangle,u.deg)).ra.degree
sn_data["dec"] = SkyCoord(sn_data["ra"], sn_data["dec"],unit=(u.hourangle,u.deg)).dec.degree
#display(sn_data["dec"] )
#sn_data["ra"] = sn_data["coords"].ra.degree
#sn_data["dec"] = sn_data["coords"].dec.degree

days_before = 20.0
days_after = 100.0
sn_data["maxdate"] = sn_data["maxdate"].str.replace('/','-')
sn_data["start_end_date"] = ''
sn_data['start_end_date'] = sn_data['start_end_date'].astype(object)
for index, row in sn_data.iterrows():
    if(str(row["maxdate"])!='nan'):
        #print("should not be nan"+ str(row["maxdate"]))
        processed_maxdate = Time(row["maxdate"], format='isot')
        #sn_data.at[index, 'start_date'] =  (processed_maxdate - TimeDelta(days_before,format='jd')).mjd  
        #sn_data.at[index, 'end_date'] = (processed_maxdate + TimeDelta(days_after, format='jd')).mjd
        start_date =  (processed_maxdate - TimeDelta(days_before,format='jd')).mjd 
        end_date = (processed_maxdate + TimeDelta(days_after, format='jd')).mjd
        array = [start_date, end_date]
        sn_data.at[index,'start_end_date'] = np.array(array) 
        
print(sn_data["start_end_date"])
        
### convert the dataframe table into an satropy table then write it into a votable xml file

table = Table.from_pandas(sn_data)     # an astropy table
votable = from_table(table)         #convert the astropy table into votable
writeto(votable, "tmp/catalog.xml")      #write the votable into the xml , need to change the string type to char, and rename the references column

#print(sn_data["start_date"])

### query content
'''
anonSubject = net.Subject()
client = CadcTapClient(anonSubject, resource_id ='ivo://cadc.nrc.ca/tap')

query ="""SELECT TOP 10* FROM tap_upload.tmptable as Tmp, caom2.Plane AS Plane 
    JOIN caom2.Observation AS Observation ON Plane.obsID = Observation.obsID
    WHERE (INTERSECTS( INTERVAL( 100.0, 200.0), Plane.time_bounds_samples) = 1 
    AND CONTAINS( POINT('ICRS', Tmp.ra, Tmp.dec), Plane.position_bounds ) = 1 
    AND LOWER(Plane.energy_bandpassName) LIKE 'r' 
    AND collection = 'CFHT'
    AND calibrationLevel >= 2
    AND (Plane.quality_flag IS NULL OR Plane.quality_flag != 'junk'))
    ORDER BY time_bounds_lower ASC"""
#INTERSECTS( INTERVAL( Tmp.start_date, Tmp.end_date ), Plane.time_bounds_samples ) = 1 AND
    
#out_buffer = BytesIO()
def_table = os.path.join('tmp','catalog.xml')
client.query(query, output_file= "tmp/out_file.csv", response_format= 'csv', tmptable='tmptable:' + def_table, timeout= 10)

### Use glue to plot
app = gj.jglue()
out_data = app.load_data('tmp/out_file.csv')

scatter_viewer = app.scatter2d(x = 'redshift',y = 'ebv', data = out_data)
histogram_viewer = app.histogram1d(x = 'redshift', data = out_data)
'''

'''

query_params = {
    'num': '',  # Restricts the number of results (empty string returns all)
    'mjd_start': start_date.mjd,
    'mjd_end': end_date.mjd,
    'ra': coords.ra.degree,
    'dec': coords.dec.degree,
    'filter': 'r',
    'collection': 'CFHT',
    'cal_level': 2,
    'order': 'ASC'  # Order the results from oldest to newest
}

query = """SELECT {num} * FROM caom2.Plane AS Plane 
    JOIN caom2.Observation AS Observation ON Plane.obsID = Observation.obsID
    WHERE (INTERSECTS( INTERVAL( {mjd_start}, {mjd_end} ), Plane.time_bounds_samples ) = 1 
    AND CONTAINS( POINT('ICRS', {ra}, {dec}), Plane.position_bounds ) = 1 
    AND LOWER(Plane.energy_bandpassName) LIKE '{filter}%' 
    AND collection = '{collection}'
    AND calibrationLevel >= {cal_level}
    AND (Plane.quality_flag IS NULL OR Plane.quality_flag != 'junk'))
    ORDER BY time_bounds_lower {order}"""
    

'''
































    

0                                 NaN
1                  [57382.0, 57502.0]
2                  [56190.0, 56310.0]
3                  [56609.0, 56729.0]
4                                 NaN
                     ...             
12433    [57648.0, 57767.99998842592]
12434    [57653.0, 57772.99998842592]
12435    [57679.0, 57798.99998842592]
12436    [57695.0, 57814.99998842592]
12437    [57708.0, 57827.99998842592]
Name: start_end_date, Length: 12438, dtype: object


TypeError: len() of unsized object